In [3]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import requests
import json
from math import radians, cos, sin, asin, acos, sqrt, pi

## Data Import

In [4]:
df_vehicles = pd.read_csv('output/df_distances.csv')
df_terminals = pd.read_excel('../data/railroad_terminals.xlsx')
df_receiver = pd.read_csv('output/df_receiver_geolocation.csv', index_col=0).dropna()
df_shipper = pd.read_csv('output/df_shipper_geolocation.csv', index_col=0)
data_nshift = pd.read_csv('../ui/app/assets/SE _ nShift _ Oct 22.csv', sep='|')

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_26472\3353786929.py:5: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_nshift = pd.read_csv('../ui/app/assets/SE _ nShift _ Oct 22.csv', sep='|')


In [106]:
data_nshift['Sender weight (kg)'] = data_nshift.apply(lambda x: x['Sender weight (kg)'].replace('.', '').replace(',', '.'), axis = 1).astype('float')

## Feature Engineering

In [7]:
df_terminals[df_terminals['Country code'].isin(df_receiver['Receiver country code'].unique())]

,City,Road,City.1,Country code,Latitude,Longitude
0,Dijon,Chemin des Étangs,21160 Perrigny-lès-Dijon,FR,47.253418,5.018262
1,Bordeaux Combiné (Naviland),Rue Radio Londres 40-45,33130 Bègles,FR,44.793087,-0.546006
2,Noisy-le-Sec,11 rue Emmanuel Arago,93130 Noisy-le-Sec,FR,48.900025,2.463407
4,Örebro,Mannatorpsvägen 1,703 63 Örebro,SE,59.288885,15.222060
5,Werra Kombi Terminal,Industriestraße 1,36269 Philippsthal,DE,50.843494,9.971003
6,Høje Taastrup,Estlands Allé 10,2630 Høje Taastrup,DK,55.636165,12.250350


In [8]:
df_terminals['key'] = 1
# cross join
# Merging df_shipper on full nshift data
df_receiver['key'] = 1
df_receiver_terminals = df_terminals[df_terminals['Country code'].isin(df_receiver['Receiver country code'].unique())]
df_receiver_terminals = pd.merge(df_receiver[['key','Receiver latitude', 'Receiver longitude']], df_receiver_terminals[['key','Latitude', 'Longitude']], on ='key').drop("key", 1)
# Merging df_shipper on full nshift data
df_shipper['key'] = 1
df_shipper_terminals = df_terminals[df_terminals['Country code'].isin(df_shipper['Shipper country code'].unique())]
df_shipper_terminals = pd.merge(df_shipper[['key','Shipper latitude', 'Shipper longitude']], df_shipper_terminals[['key','Latitude', 'Longitude']], on ='key').drop("key", 1)

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_3580\3641398587.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_receiver_terminals = pd.merge(df_receiver[['key','Receiver latitude', 'Receiver longitude']], df_receiver_terminals[['key','Latitude', 'Longitude']], on ='key').drop("key", 1)
C:\Users\SESA667490\AppData\Local\Temp\ipykernel_3580\3641398587.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_shipper_terminals = pd.merge(df_shipper[['key','Shipper latitude', 'Shipper longitude']], df_shipper_terminals[['key','Latitude', 'Longitude']], on ='key').drop("key", 1)


In [9]:
# Merging only same country to limit calculation for now
# Merging df_shipper on full nshift data
df_receiver_terminals = pd.merge(df_receiver[['Receiver country code', 'Receiver latitude', 'Receiver longitude']], df_terminals[[
                                 'Country code', 'Latitude', 'Longitude']], left_on='Receiver country code', right_on='Country code')
df_shipper_terminals = pd.merge(df_shipper[['Shipper country code', 'Shipper latitude', 'Shipper longitude']], df_terminals[[
                                 'Country code', 'Latitude', 'Longitude']], left_on='Shipper country code', right_on='Country code')

In [10]:
def get_distance(x, name):
    # call the OSMR API
    r = requests.get(
        f"http://router.project-osrm.org/route/v1/car/{x[f'{name} longitude']},{x[f'{name} latitude']};{x['Longitude']},{x['Latitude']}?overview=false""")
    # then you load the response using the json libray
    # by default you get only one alternative so you access 0-th element of the `routes`
    routes = json.loads(r.content)
    try: 
        route_1 = routes.get("routes")[0]
        return route_1['distance']
    except:
        return None

In [95]:
df_shipper_terminals['distance'] = df_shipper_terminals.apply(lambda x: get_distance(x, 'Shipper'), axis=1)

In [96]:
df_receiver_terminals['distance'] = df_receiver_terminals.apply(lambda x: get_distance(x, 'Receiver'), axis=1)

In [11]:
df_terminal_terminals = pd.merge(df_terminals[['key','Latitude', 'Longitude']], df_terminals[['key','Latitude', 'Longitude']], on ='key').drop("key", 1)

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_3580\3279413240.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_terminal_terminals = pd.merge(df_terminals[['key','Latitude', 'Longitude']], df_terminals[['key','Latitude', 'Longitude']], on ='key').drop("key", 1)


In [106]:
df_shipper_terminals.to_csv('output/df_shipper_terminals.csv', index = False)
df_receiver_terminals.to_csv('output/df_receiver_terminals.csv', index = False)

In [12]:
co2_transport = 67.2 # THG-Emissionen (WTW) in g CO2e je tkm
co2_rail = 18.5

In [108]:
df_shipper_terminals['gCO2/t'] = (df_shipper_terminals['distance']/1000)*co2_transport

In [112]:
df_vehicles['gCO2/t'] = (df_vehicles['distance']/1000)*co2_transport

In [131]:
x = df_vehicles.iloc[0]
# call the OSMR API
r = requests.get(
    f"http://router.project-osrm.org/route/v1/car/{x['Shipper longitude']},{x['Shipper latitude']};{x['Receiver longitude']},{x['Receiver latitude']}?overview=false""")
# then you load the response using the json libray
# by default you get only one alternative so you access 0-th element of the `routes`
routes = json.loads(r.content)

In [132]:
routes

{'code': 'Ok',
 'routes': [{'legs': [{'steps': [],
     'summary': '',
     'weight': 21790.1,
     'duration': 21725.9,
     'distance': 567223.1}],
   'weight_name': 'routability',
   'weight': 21790.1,
   'duration': 21725.9,
   'distance': 567223.1}],
 'waypoints': [{'hint': 'YPcdjf___38qAAAA8wAAAD4AAACrAAAA3b7jQeSGBUMBNCVCj-fjQioAAADzAAAAPgAAAKsAAADn9AAAHf4RALrF6wIk_hEAt8XrAgEAjw3OuOqC',
   'distance': 0.61106324,
   'name': 'Rue Roland Garros',
   'location': [1.179165, 49.00601]},
  {'hint': 'hfojgHrXwogCAAAADwAAAAAAAABBAAAAjx7pP6_UFEEAAAAAgoQuQgIAAAAPAAAAAAAAAEEAAADn9AAAuj9MAI7SuQLAP0wAodK5AgAAnxHOuOqC',
   'distance': 2.162809729,
   'name': 'Rue de la République',
   'location': [4.99705, 45.732494]}]}

In [12]:
df_enriched = df_vehicles.merge(df_receiver, how='left', on=['Receiver longitude', 'Receiver latitude']).merge(
    df_shipper, how='left', on=['Shipper longitude', 'Shipper latitude']).merge(data_nshift, how='left', on=['Receiver address',
                                                                                                               'Receiver city', 'Receiver country code', 'Receiver zip',
                                                                                                               'Shipper address', 'Shipper city', 'Shipper country code',
                                                                                                               'Shipper zip'])


In [15]:
data_nshift

,Shipper,Shipper country code,Shipper zip,Shipper city,Shipper address,Order number,Shipment number,Receiver,Receiver country code,Receiver zip,...,Carrier,Product,Sender weight (kg),Number of packages,Pickup date,Services,Radar status,Status category,Status,Carrier TT date time stamp;
0,Evreux,FR,27900,Guichainville,Rue Roland Garos,879233105,"2,50076E+14",AFCA GENAS,FR,69747,...,DPD France (FR),DPD Classic (B2B),18.824,3,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,25/10/2022 11:30:00;
1,Evreux,FR,27900,Guichainville,Rue Roland Garos,879227036,"2,50076E+14",ELIWELL FRANCE SA,FR,92390,...,DPD France (FR),DPD Classic (B2B),20.044,3,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,25/10/2022 10:51:00;
2,Evreux,FR,27900,Guichainville,Rue Roland Garos,879232085,"2,50076E+14",SERT,FR,69150,...,DPD France (FR),DPD Classic (B2B),0.306,1,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,27/10/2022 11:01:00;
3,Evreux,FR,27900,Guichainville,Rue Roland Garos,879232252,"2,50076E+14",OET,FR,35830,...,DPD France (FR),DPD Classic (B2B),2.072,1,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,25/10/2022 10:44:00;
4,Evreux,FR,27900,Guichainville,Rue Roland Garos,879232023,"2,50076E+14",YESSS DECINES CHARPIEU,FR,69150,...,DPD France (FR),DPD Classic (B2B),0.99,1,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,25/10/2022 09:39:00;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71774,RDC Örebro,SE,70227,Örebro,Lastgatan 2,3358831,40170726205850928847,FORENEDE MONTØRER AS,NO,3610,...,Bring (NO),Stykkgods,"13,169",3,44839,NaN,Delivered,Delivered,Delivered,"44840,48419"
71775,RDC Örebro,SE,70227,Örebro,Lastgatan 2,3358812,DO742104,Dansk Tavle Teknik A/S,DK,3000,...,Danske Fragtmænd (DK),Palle/Enhedsforsendelse,"1,3",1,44839,NaN,Delivered,Delivered,Delivered,"44840,50556"
71776,RDC Örebro,SE,70227,Örebro,Lastgatan 2,3358832,DO742105,Scanel International A/S,DK,9900,...,Danske Fragtmænd (DK),Palle/Enhedsforsendelse,"0,1",1,44839,NaN,Delivered,Delivered,Delivered,"44840,44861"
71777,RDC Örebro,SE,70227,Örebro,Lastgatan 2,3358789,DO742106,Schneider Electric A/S,DK,5220,...,Danske Fragtmænd (DK),Palle/Enhedsforsendelse,"2,482",1,44839,NaN,Delivered,Delivered,Delivered,"44840,37569"


In [17]:
data_nshift.merge(df_receiver, how='left', on=['Receiver address',
                                     'Receiver city', 'Receiver country code', 'Receiver zip'])


,Shipper,Shipper country code,Shipper zip,Shipper city,Shipper address,Order number,Shipment number,Receiver,Receiver country code,Receiver zip,...,Sender weight (kg),Number of packages,Pickup date,Services,Radar status,Status category,Status,Carrier TT date time stamp;,Receiver latitude,Receiver longitude
0,Evreux,FR,27900,Guichainville,Rue Roland Garos,879233105,"2,50076E+14",AFCA GENAS,FR,69747,...,18.824,3,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,25/10/2022 11:30:00;,45.732513,4.997056
1,Evreux,FR,27900,Guichainville,Rue Roland Garos,879227036,"2,50076E+14",ELIWELL FRANCE SA,FR,92390,...,20.044,3,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,25/10/2022 10:51:00;,48.944825,2.310601
2,Evreux,FR,27900,Guichainville,Rue Roland Garos,879232085,"2,50076E+14",SERT,FR,69150,...,0.306,1,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,27/10/2022 11:01:00;,45.748306,4.939595
3,Evreux,FR,27900,Guichainville,Rue Roland Garos,879232252,"2,50076E+14",OET,FR,35830,...,2.072,1,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,25/10/2022 10:44:00;,48.162535,-1.648912
4,Evreux,FR,27900,Guichainville,Rue Roland Garos,879232023,"2,50076E+14",YESSS DECINES CHARPIEU,FR,69150,...,0.99,1,24/10/2022 0:00,NaN,Delivered,Delivered,Delivered,25/10/2022 09:39:00;,45.766314,4.943846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71774,RDC Örebro,SE,70227,Örebro,Lastgatan 2,3358831,40170726205850928847,FORENEDE MONTØRER AS,NO,3610,...,"13,169",3,44839,NaN,Delivered,Delivered,Delivered,"44840,48419",NaN,NaN
71775,RDC Örebro,SE,70227,Örebro,Lastgatan 2,3358812,DO742104,Dansk Tavle Teknik A/S,DK,3000,...,"1,3",1,44839,NaN,Delivered,Delivered,Delivered,"44840,50556",NaN,NaN
71776,RDC Örebro,SE,70227,Örebro,Lastgatan 2,3358832,DO742105,Scanel International A/S,DK,9900,...,"0,1",1,44839,NaN,Delivered,Delivered,Delivered,"44840,44861",NaN,NaN
71777,RDC Örebro,SE,70227,Örebro,Lastgatan 2,3358789,DO742106,Schneider Electric A/S,DK,5220,...,"2,482",1,44839,NaN,Delivered,Delivered,Delivered,"44840,37569",NaN,NaN


In [13]:
df_enriched.shape

(29894, 27)

In [1]:
df_vehicles.shape()

NameError: name 'df_vehicles' is not defined

In [117]:
df_enriched['gCO2/t']

0        38117.39232
1        38117.39232
2        38117.39232
3        38117.39232
4        38117.39232
            ...     
29875    54908.34048
29876    54908.34048
29877    54908.34048
29878    54908.34048
29879    54908.34048
Name: gCO2/t, Length: 29880, dtype: float64

In [119]:
df_enriched['kgCO2'] = (df_enriched['gCO2/t'] * (df_enriched['Sender weight (kg)']/1000))/1000

In [129]:
round((df_enriched[df_enriched['distance']<200000]['kgCO2'].sum()/df_enriched['kgCO2'].sum())*100, 2)

6.33

In [121]:
df_enriched['kgCO2'].sum()

21311553.882511526

In [111]:
df_receiver_terminals['gCO2/t'] = (df_receiver_terminals['distance']/1000)*co2_transport

In [13]:
df_terminal_terminals = df_terminal_terminals[(df_terminal_terminals['Latitude_x']!=df_terminal_terminals['Latitude_y'])&(df_terminal_terminals['Longitude_x']!=df_terminal_terminals['Longitude_y'])]

In [17]:
def calculate_spherical_distance(x, r=6371):
    # Convert degrees to radians
    coordinates = x['Latitude_x'], x['Longitude_x'], x['Latitude_y'], x['Longitude_y']
    phi1, lambda1, phi2, lambda2 = [
        radians(c) for c in coordinates
    ]
    
    # Apply the haversine formula
    d = r*acos(cos(phi2-phi1) - cos(phi1) * cos(phi2) *
              (1-cos(lambda2-lambda1)))
    return d

In [22]:
df_terminal_terminals['distance'] = df_terminal_terminals.apply(lambda x: calculate_spherical_distance(x), axis = 1)

## Algorithms

In [2]:
def dijkstra(current, nodes, distances):
    # These are all the nodes which have not been visited yet
    unvisited = {node: None for node in nodes}
    # It will store the shortest distance from one node to another
    visited = {}
    # It will store the predecessors of the nodes
    currentDistance = 0
    unvisited[current] = currentDistance
    # Running the loop while all the nodes have been visited
    while True:
        # iterating through all the unvisited node
        for neighbour, distance in distances[current].items():
            # Iterating through the connected nodes of current_node (for 
            # example, a is connected with b and c having values 10 and 3
            # respectively) and the weight of the edges
            if neighbour not in unvisited: continue
            newDistance = currentDistance + distance
            if unvisited[neighbour] is None or unvisited[neighbour] > newDistance:
                unvisited[neighbour] = newDistance
        # Till now the shortest distance between the source node and target node 
        # has been found. Set the current node as the target node
        visited[current] = currentDistance
        del unvisited[current]
        if not unvisited: break
        candidates = [node for node in unvisited.items() if node[1]]
        print(sorted(candidates, key = lambda x: x[1]))
        current, currentDistance = sorted(candidates, key = lambda x: x[1])[0]
    return visited
  
nodes = ('A', 'B', 'C', 'D', 'E')
distances = {
    'A': {'B': 5, 'C': 2},
    'B': {'C': 2, 'D': 3},
    'C': {'B': 3, 'D': 7, 'E': 3},
    'D': {'E': 7},
    'E': {'D': 9}}
current = 'A'
  
print(dijkstra(current, nodes, distances))

[('C', 2), ('B', 5)]
[('B', 5), ('E', 5), ('D', 9)]
[('E', 5), ('D', 8)]
[('D', 8)]
{'A': 0, 'C': 2, 'B': 5, 'E': 5, 'D': 8}


## Code from: 

In [3]:
def shortest_path(graph, node1, node2):
    path_list = [[node1]]
    path_index = 0
    # To keep track of previously visited nodes
    previous_nodes = {node1}
    if node1 == node2:
        return path_list[0]
        
    while path_index < len(path_list):
        current_path = path_list[path_index]
        last_node = current_path[-1]
        next_nodes = graph[last_node]
        # Search goal node
        if node2 in next_nodes:
            current_path.append(node2)
            return current_path
        # Add new paths
        for next_node in next_nodes:
            if not next_node in previous_nodes:
                new_path = current_path[:]
                new_path.append(next_node)
                path_list.append(new_path)
                # To avoid backtracking
                previous_nodes.add(next_node)
        # Continue to next path in list
        path_index += 1
    # No path is found
    return []

In [6]:
graph = {}
graph[1] = {2, 5}
graph[2] = {1, 3, 5}
graph[3] = {2, 4}
graph[4] = {3, 5, 6}
graph[5] = {1, 2, 4}
graph[6] = {4}

In [7]:
shortest_path(graph, 1, 6)

[1, 5, 4, 6]

Setting up graph: 
* Nodes clients and railroad terminals
* Road distance: Using Google 
* Rail distance: Using straight line for now? 
* looking for time, distance, emissions